# GCSH individuals

In [10]:
import pyphetools
from pyphetools.visualization import IndividualTable, QcVisualizer
from IPython.display import display, HTML
from pyphetools.creation import TemplateImporter
template = "input/GCSH_individuals.xlsx"
hp_json = "../hp.json"
created_by = "0000-0002-0736-9199"
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.63


In [8]:
timporter = TemplateImporter(template=template, hp_json=hp_json, created_by=created_by)

In [9]:
individuals, cvalidator = timporter.import_phenopackets_from_template()

HPO version 2024-02-27
Created encoders for 30 fields
 we got OMIM:620423  NM_004483.5
We output 3 GA4GH phenopackets to the directory phenopackets


In [11]:
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,2


In [12]:
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Proband 1 (MALE; P3Y),Multiple mitochondrial dysfunctions syndrome 7 (OMIM:620423),NM_004483.5:c.1A>G (homozygous),P1M: Lethargy (HP:0001254); Feeding difficulties (HP:0011968); Myoclonic seizure (HP:0032794)P3Y: Secondary microcephaly (HP:0005484); Increased CSF glycine concentration (HP:0500230); Hyperglycinemia (HP:0002154); Elevated brain lactate level by MRS (HP:0012707); Elevated brain glycine level by MRS (HP:0034893)Global developmental delay (HP:0001263); Dolichocephaly (HP:0000268); Anteverted nares (HP:0000463); Open mouth (HP:0000194); Gingival overgrowth (HP:0000212); Hypertonia (HP:0001276); Ankle clonus (HP:0011448); excluded: Hypotonia (HP:0001252); excluded: Hypernatremia (HP:0003228); excluded: Metabolic acidosis (HP:0001942)
Proband 2 (FEMALE; P1Y1M),Multiple mitochondrial dysfunctions syndrome 7 (OMIM:620423),NM_004483.5:c.1A>G (homozygous),P2D: Lethargy (HP:0001254); Feeding difficulties (HP:0011968); Hypotonia (HP:0001252); Seizure (HP:0001250)P3M: Elevated brain lactate level by MRS (HP:0012707); Elevated brain glycine level by MRS (HP:0034893)Hypernatremia (HP:0003228); Metabolic acidosis (HP:0001942); Global developmental delay (HP:0001263)
Proband 3 (FEMALE; P6D),Multiple mitochondrial dysfunctions syndrome 7 (OMIM:620423),NM_004483.5:c.1A>G (homozygous),P3D: Lethargy (HP:0001254)Poor suck (HP:0002033); Partial atrioventricular canal defect (HP:0011577)


In [14]:
#df = timporter.create_hpoa_from_phenopackets(pmid="PMID:33890291", moi="Autosomal recessive")